# Input pipeline into Keras

In this notebook, we will look at how to read large datasets, datasets that may not fit into memory, using TensorFlow. We can use the tf.data pipeline to feed data to Keras models that use a TensorFlow backend.

## Learning Objectives
1. Use tf.data to read CSV files
2. Load the training data into memory
3. Prune the data by removing columns
4. Use tf.data to map features and labels
5. Adjust the batch size of our dataset
6. Shuffle the dataset to optimize for deep learning

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](../labs/input_pipeline.ipynb) -- try to complete that notebook first before reviewing this solution notebook. 

Let's start off with the Python imports that we need.

In [ ]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

In [ ]:
%%bash
!pip install tensorflow==2.1.0 --user

Let's make sure we install the necessary version of tensorflow. After doing the pip install above, click __Restart the kernel__ on the notebook so that the Python environment picks up the new packages.

In [19]:
import os, json, math
import numpy as np
import shutil
import logging
# SET TF ERROR LOG VERBOSITY
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import tensorflow as tf

print("TensorFlow version: ",tf.version.VERSION)

PROJECT = "your-gcp-project-here" # REPLACE WITH YOUR PROJECT NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["BUCKET"] = PROJECT # DEFAULT BUCKET WILL BE PROJECT ID

if PROJECT == "your-gcp-project-here":
  print("Don't forget to update your PROJECT name! Currently:", PROJECT)

TensorFlow version:  2.1.0


In [27]:
# If you're not using TF 2.0+, let's enable eager execution
if tf.version.VERSION < '2.0':
    print('Enabling v2 behavior and eager execution; if necessary restart kernel, and rerun notebook')
    tf.enable_v2_behavior()

## Locating the CSV files

We will start with the CSV files that we wrote out in the [first notebook](../01_explore/taxifare.iypnb) of this sequence. Just so you don't have to run the notebook, we saved a copy in ../data

In [13]:
!ls -l ../../data/*.csv

-rw-r--r-- 1 jupyter jupyter 123590 Sep 12 16:33 ../data/taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 579055 Sep 12 16:33 ../data/taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 123114 Sep 12 16:33 ../data/taxi-valid.csv


## Use tf.data to read the CSV files

See the documentation for [make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset).
If you have TFRecords (which is recommended), use [make_batched_features_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_batched_features_dataset) instead.

In [21]:
CSV_COLUMNS  = ['fare_amount',  'pickup_datetime',
                'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 
                'passenger_count', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS     = [[0.0],['na'],[0.0],[0.0],[0.0],[0.0],[0.0],['na']]

In [22]:
# load the training data
def load_dataset(pattern):
  return tf.data.experimental.make_csv_dataset(pattern, 1, CSV_COLUMNS, DEFAULTS)

tempds = load_dataset('../../data/taxi-train*')
print(tempds)

<PrefetchDataset shapes: OrderedDict([(fare_amount, (1,)), (pickup_datetime, (1,)), (pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,)), (key, (1,))]), types: OrderedDict([(fare_amount, tf.float32), (pickup_datetime, tf.string), (pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32), (key, tf.string)])>


Note that this is a prefetched dataset. If you loop over the dataset, you'll get the rows one-by-one. Let's convert each row into a Python dictionary:

In [23]:
# print a few of the rows
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    print(n, row_data)
    if n > 2:
        break

0 {'pickup_datetime': array([b'2012-08-10 12:17:02 UTC'], dtype=object), 'pickup_longitude': array([-73.98314], dtype=float32), 'pickup_latitude': array([40.741684], dtype=float32), 'key': array([b'460'], dtype=object), 'fare_amount': array([6.1], dtype=float32), 'dropoff_longitude': array([-73.995285], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'dropoff_latitude': array([40.744015], dtype=float32)}
1 {'pickup_datetime': array([b'2014-09-10 07:23:17 UTC'], dtype=object), 'pickup_longitude': array([-73.99419], dtype=float32), 'pickup_latitude': array([40.745983], dtype=float32), 'key': array([b'206'], dtype=object), 'fare_amount': array([5.], dtype=float32), 'dropoff_longitude': array([-73.98535], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'dropoff_latitude': array([40.753426], dtype=float32)}
2 {'pickup_datetime': array([b'2013-04-18 08:48:00 UTC'], dtype=object), 'pickup_longitude': array([-73.864914], dtype=float32), 'pickup_latitude': array([4

What we really need is a dictionary of features + a label. So, we have to do two things to the above dictionary. (1) remove the unwanted column "key" and (2) keep the label separate from the features.

In [24]:
# get features, label
def features_and_labels(row_data):
    for unwanted_col in ['pickup_datetime', 'key']:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label  # features, label

# print a few rows to make it sure works
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    features, label = features_and_labels(row_data)
    print(n, label, features)
    if n > 2:
        break

0 [6.1] {'pickup_longitude': array([-73.98314], dtype=float32), 'pickup_latitude': array([40.741684], dtype=float32), 'dropoff_longitude': array([-73.995285], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'dropoff_latitude': array([40.744015], dtype=float32)}
1 [5.] {'pickup_longitude': array([-73.99419], dtype=float32), 'pickup_latitude': array([40.745983], dtype=float32), 'dropoff_longitude': array([-73.98535], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'dropoff_latitude': array([40.753426], dtype=float32)}
2 [6.] {'pickup_longitude': array([-73.864914], dtype=float32), 'pickup_latitude': array([40.771378], dtype=float32), 'dropoff_longitude': array([-73.86421], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'dropoff_latitude': array([40.77], dtype=float32)}
3 [5.] {'pickup_longitude': array([-73.96325], dtype=float32), 'pickup_latitude': array([40.774113], dtype=float32), 'dropoff_longitude': array([-73.95772], dtype=float32), 'pa

## Batching

Let's do both (loading, features_label)
in our load_dataset function, and also add batching.

In [25]:
def load_dataset(pattern, batch_size):
  return (
      tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
  )

# try changing the batch size and watch what happens.
tempds = load_dataset('../../data/taxi-train*', batch_size=2)
print(list(tempds.take(3))) # truncate and print as a list 

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=419, shape=(2,), dtype=float32, numpy=array([-73.976524, -73.999306], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=418, shape=(2,), dtype=float32, numpy=array([40.756996, 40.73008 ], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=416, shape=(2,), dtype=float32, numpy=array([-73.94917, -73.98662], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=415, shape=(2,), dtype=float32, numpy=array([40.777275, 40.728127], dtype=float32)>), ('passenger_count', <tf.Tensor: id=417, shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: id=420, shape=(2,), dtype=float32, numpy=array([14. ,  5.3], dtype=float32)>), (OrderedDict([('pickup_longitude', <tf.Tensor: id=425, shape=(2,), dtype=float32, numpy=array([-73.97756, -73.97157], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=424, shape=(2,), dtype=float32, numpy=array([40.74649, 40.75433], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=422

## Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

In [26]:
def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
  dataset = (tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
             .cache())
  if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.prefetch(1) # take advantage of multi-threading; 1=AUTOTUNE
  return dataset

tempds = load_dataset('../../data/taxi-train*', 2, tf.estimator.ModeKeys.TRAIN)
print(list(tempds.take(1)))
tempds = load_dataset('../../data/taxi-valid*', 2, tf.estimator.ModeKeys.EVAL)
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=552, shape=(2,), dtype=float32, numpy=array([-74.00672, -73.78672], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=551, shape=(2,), dtype=float32, numpy=array([40.741623, 40.641136], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=549, shape=(2,), dtype=float32, numpy=array([-73.9633 , -73.78232], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=548, shape=(2,), dtype=float32, numpy=array([40.762768, 40.648724], dtype=float32)>), ('passenger_count', <tf.Tensor: id=550, shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: id=553, shape=(2,), dtype=float32, numpy=array([17.5,  7.7], dtype=float32)>)]
[(OrderedDict([('pickup_longitude', <tf.Tensor: id=655, shape=(2,), dtype=float32, numpy=array([-73.98504, -73.99718], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=654, shape=(2,), dtype=float32, numpy=array([40.76108 , 40.714516], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=65

In the next notebook, we will build the model using this input pipeline.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.